In [1]:
# Import Necessary Datasets (added comments to the whole code)
import numpy as np
import pandas as pd
import cv2
from skimage.feature import hog
from sklearn.datasets import fetch_openml
from scipy import ndimage
from skimage.measure import label, regionprops
from skimage import morphology
import matplotlib.pyplot as plt
from skimage import morphology, measure
from skimage.transform import probabilistic_hough_line
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Import the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(int)

print(f"{X.shape}")
print(f"{y.shape}")

(70000, 784)
(70000,)


In [3]:
# Skeletonization
def skeletonize_image(image):
    binary_image = image > np.mean(image)
    
    skeleton = morphology.skeletonize(binary_image)
    
    return skeleton

In [4]:
# Number of Dark Pixels
def num_dark_pixels(image):
    return np.sum(image < 128)

In [5]:
# Vertical Symmetry
def vertical_symmetry(image):
    width = image.shape[1]
    left_half = image[:, :width//2]
    right_half = image[:, width//2:]
    right_half_flipped = np.flip(right_half, axis=1)
    return np.sum(left_half == right_half_flipped) / left_half.size

In [6]:
# Horizontal Symmetry
def horizontal_symmetry(image):
    height = image.shape[0]
    top_half = image[:height//2, :]
    bottom_half = image[height//2:, :]
    bottom_half_flipped = np.flip(bottom_half, axis=0)
    return np.sum(top_half == bottom_half_flipped) / top_half.size

In [7]:
# Number of Corners
def compute_corners(image):
    corners = cv2.cornerHarris(image.astype(np.float32), blockSize=2, ksize=3, k=0.04)
    corners = cv2.dilate(corners, None)
    corner_count = np.sum(corners > 0.01 * corners.max())
    return corner_count

In [8]:
# Number of Intersections
def compute_intersections(image):
    binary_image = image > np.mean(image)
    skeleton = morphology.skeletonize(binary_image)
    labeled_image = label(skeleton)
    intersections = 0
    for region in regionprops(labeled_image):
        if region.area > 3:
            intersections += 1
    
    return intersections

In [9]:
# Aspect Ratio
def compute_aspect_ratio(image):
    _, _, width, height = cv2.boundingRect(image)
    aspect_ratio = width / float(height)
    return aspect_ratio

In [10]:
# Enclosed Areas
def compute_enclosed_areas(image):
    labeled_image = label(image)
    regions = regionprops(labeled_image)
    enclosed_areas = len([region for region in regions if region.area > 100]) 
    return enclosed_areas

In [11]:
# Straight Lines
def compute_straight_lines(image):
    lines = probabilistic_hough_line(image, threshold=10, line_length=5, line_gap=3)
    line_count = len(lines)
    return line_count

In [12]:
# This function extracts the features from the images shown in the MNIST dataset
def extract_features(X):
    features = []
    for idx in range(len(X)):
        image = X.iloc[idx].values.reshape(28, 28).astype(np.uint8)
        
        dark_pixels = num_dark_pixels(image)
        vert_sym = vertical_symmetry(image)
        horiz_sym = horizontal_symmetry(image)
        aspect_ratio = compute_aspect_ratio(image)
        corner_count = compute_corners(image)
        intersections = compute_intersections(image)
        line_count = compute_straight_lines(image)
        enclosed_areas = compute_enclosed_areas(image)
        
        features.append([dark_pixels, vert_sym, horiz_sym, aspect_ratio, corner_count, intersections, line_count, enclosed_areas])
    return features


In [13]:
# This final cell splits the data into training and testing, and then stores extracted features in a dataframe for X and y training data. 
# An average feature vector for digits 0-9 is created by finding an average of each features for each digit
# Weights are then added to the features and then to the average feature vector for each digit
# The features are then extracted for testing data with weights and stored in another dataframe
# The distance (Euclidean Distance) is calculated between the actual features of the image and the average feature vector
# To do this, I created a function called "classify_image" and added the weight changes to it (Had to learn what 'linalg' does for Euclidean distance)
# Finally, by applying "classify_image" and converting a row from the testing dataframe into an array using Numpy, I was able to convert all the rows into a series in 'y_pred'
# The accuracy score was then calculated in percentage
feature_weights = np.array([1, 1.1, 1.1, 1, 1, 1, 1, 1])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000000)

train_features = extract_features(X_train)
train_df = pd.DataFrame(train_features, columns=['num_dark_pixels', 'vertical_symmetry', 'horizontal_symmetry', 'aspect_ratio', 'corners', 'intersections', 'straight_lines', 'enclosed_areas'])
train_df['label'] = y_train.values

average_feature_vectors = train_df.groupby('label').mean()

test_features = extract_features(X_test)
test_df = pd.DataFrame(test_features, columns=['num_dark_pixels', 'vertical_symmetry', 'horizontal_symmetry', 'aspect_ratio', 'corners', 'intersections', 'straight_lines', 'enclosed_areas'])

def classify_image(image_features, average_feature_vectors, feature_weights):
    weighted_average_vectors = average_feature_vectors.values * feature_weights
    weighted_image_features = image_features * feature_weights
    distances = np.linalg.norm(weighted_average_vectors - weighted_image_features, axis=1)
    return average_feature_vectors.index[np.argmin(distances)]

y_pred = test_df.apply(lambda row: classify_image(row.values, average_feature_vectors, feature_weights), axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 36.98%


In [14]:
X_train.shape

(56000, 784)

In [15]:
y_train.shape

(56000,)

In [16]:
X_test.shape

(14000, 784)

In [17]:
y_test.shape

(14000,)

In [18]:
# End of Program